<a href="https://colab.research.google.com/github/jimmy-pink/colab-playground/blob/main/pre-trained/EfficientNetB3-FolderIconRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用EfficientNetB3微调以解决FolderIcon二分类问题



### 问题分析
#### 背景
已在VGG16预训练模型上微调，在对2170多个folder icon预测，存在以下问题：
- 发现准确率仅为88%-91%
- 更要命的是基于VGG16训练出来的模型经常犯超低级错误，将毫不相干的图像判断为真。   

鉴于VGG16是非常老的模型，现以EfficientNetB3再训练一版，观察模型性能有无明显提升。



#### 要求

|**特性**|**说明**|
|---|---|
|更深、更窄、更复杂|EfficientNetB3 用更少参数提取更细粒度的特征，对训练数据的细节更敏感|
|训练更精细|它倾向于在微小差异中学习（如纹理、边角等视觉细节），对标签的一致性要求更高|
|参数更少，欠拟合风险更大|如果标签混乱，EfficientNet 可能“学不到明确的规律”，而不是像 VGG 一样粗暴记忆|


需要对标签重新处理，去掉模糊边界的样本，给任务规划清晰的边界。  
二分类问题：
- 是：标准的文件夹图标
- 否：
  - 使用预训练模型（如 EfficientNetB0）提取所有图标的特征向量。
  - 计算每张图与“正样本平均特征”之间的相似度（余弦距离）。
  - 选择距离最远的 N 张图像，作为“最不像文件夹”的负样本。

In [24]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam

### 数据准备

In [2]:
## 转存 https://drive.google.com/drive/folders/1xwtf91GSyeIc7ohpCKsDCYv3zXgKa0sf
from google.colab import drive
drive.mount('/content/drive')

# 挂载 Google Drive
base_dir = '/content/drive/MyDrive/Google AI Studio/data/folder-icon-images/'  # Google Drive 中的图像文件夹路径
train_dir=base_dir+'is_folder'
drive_train_validate_dir = base_dir + "train_validate"

Mounted at /content/drive


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 设置数据增强
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)

# 使用 flow_from_directory 加载训练数据
# 数据增强 + 归一化
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.3  # 30% 作为验证集
)

# 训练集生成器
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary',  # 二分类用 binary，多分类用 categorical
    subset='training'  # 指定是训练集
)

# 验证集生成器
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # 指定是验证集
)
print(f"找到的训练样本数: {train_generator.samples}")
print(f"找到的验证样本数: {validation_generator.samples}")
images, labels = next(train_generator)
print("图像形状:", images.shape)
print("标签形状:", labels.shape)

Found 799 images belonging to 2 classes.
Found 340 images belonging to 2 classes.
找到的训练样本数: 799
找到的验证样本数: 340
图像形状: (32, 300, 300, 3)
标签形状: (32,)


In [15]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
# 获取训练集的真实标签（需确保 train_generator.classes 是整数标签 0/1）
y_train = train_generator.classes
# 计算类别权重（classes 需是 NumPy 数组）
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),  # 自动提取唯一类别（如 [0, 1]）
    y=y_train
)
# 转换为字典格式
class_weights = {i: weight for i, weight in enumerate(class_weights)}
print("类别权重:", class_weights)

类别权重: {0: np.float64(0.9603365384615384), 1: np.float64(1.0430809399477807)}


### 建模与训练

In [25]:
base_model = EfficientNetB2(include_top=False, weights='imagenet', input_shape=(300, 300, 3))

# 冻结前 n% 的层
freeze_until = int(len(base_model.layers) * 0.9)
for i, layer in enumerate(base_model.layers):
    layer.trainable = i >= freeze_until

31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import regularizers

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.1)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

optimizer = AdamW(learning_rate=1e-4, weight_decay=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
from tensorflow.keras.callbacks import Callback
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') >= 0.98 and logs.get('val_loss') < 0.2 ):
            self.model.stop_training = True
callbacks = myCallback()
# 训练模型
history = model.fit(train_generator,
          epochs=30,
          validation_data=validation_generator,
          callbacks=[callbacks])

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 115s 3s/step - accuracy: 0.5127 - loss: 23.5546 - val_accuracy: 0.5206 - val_loss: 21.6653
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 11s 424ms/step - accuracy: 0.4984 - loss: 21.0764 - val_accuracy: 0.5206 - val_loss: 19.3562
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 0.5204 - loss: 18.8351

KeyboardInterrupt: 